# Práca s dátami

Kvalita a množstvo dostupných dát je najkritickejšou zložkou pri úspešnom trénovaní hlbokých neurónových sietí, aj preto témou dnešného cvičenia je ich predspracovanie a príprava na použitie pri učení modelov. Pozrieme sa na možné prístupy pri práci s chýbajúcimi údajmi, na úpravu kategorických dát do podoby, ktorá je spracovateľná neurónovými sieťami, a na výber najdôležitejších príznakov pre trénovanie.

## Chýbajúce údaje

V reálnych aplikáciách sa často stáva, že tréningovým príkladom chýbajú niektoré hodnoty z rôznych dôvodov. Môže sa jednať o chybu pri zbere dát, neaplikovateľnosť niektorých meraní alebo jednoducho nevyplnené miesta v dotazníku. Väčšina modelov nedokáže s chýbajúcimi hodnotami pracovať alebo tie by mohli priniesť nepredvídateľné výsledky, a preto je nevyhnutné zaoberať sa chýbajúcimi hodnotami ešte pred trénovaním.

In [ ]:
import pandas as pd
from io import StringIO
csv_data = '''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''
df = pd.read_csv(StringIO(csv_data))
df

Možné prístupy pri spracovaní chýbajúcich hodnôt sú eliminácia príkladov a náhrada chýbajúcich hodnôt (napríklad priemerom atribútu).

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(axis=0)  # mazanie riadkov
# df.dropna(axis=1)  # mazanie stĺpcov
# df.dropna(how='all')  # mazanie iba riadkov v ktorých chýbajú všetky hodnoty
# df.dropna(thresh=4)  # mazanie riadkov s menej ako 4 hodnotami
# df.dropna(subset=['C'])  # mazanie riadkov, v ktorých chýba hodnota atribútu C

In [ ]:
from sklearn.impute import SimpleImputer
import numpy as np
imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data

# alebo:
# df.fillna(df.mean())

## Spracovanie kategorických údajov

V datasetoch často nájdete aj nečíselné premenné, ktoré môžu nadobudnúť iba diskrétne hodnoty, reprezentujú teda kategorické dáta. Tie môžu byť ordinálne alebo nominálne. Ordinálne údaje môžeme zoradiť (napríklad veľkosť trička), kým nominálne nenaznačujú žiadne poradie (napríklad farba trička). Okrem toho často potrebujete transformovať aj atribút reprezentujúci triedu.

In [ ]:
import pandas as pd
df = pd.DataFrame([
    ['green', 'M', 10.1, 'class2'],
    ['red', 'L', 13.5, 'class1'],
    ['blue', 'XL', 15.3, 'class2']])
df.columns = ['color', 'size', 'price', 'classlabel']
df

V prípade ordinálnych príznakov ich môžeme nahradiť číselnými hodnotami z istej postupnosti, napríklad pre veľkosť tričiek môžeme použiť mapovanie: *M = 1, L = 2, XL = 3*:

In [ ]:
size_mapping = { 'XL': 3, 'L': 2, 'M': 1 }
df['size'] = df['size'].map(size_mapping)
df

**Poznámka:** Pre ordinálne hodnoty sa niekedy používa enkódovanie na základe prahovej hodnoty, napríklad veľkosť trička by sme vedeli reprezentovať ako dvojicu hodnôt, kde prvá vyjadruje, či veľkosť je väčšia ako M, kým druhá hodnota vyjadruje, či je tričko väčšie ako L.

Automatické priradenie hodnôt si môžeme ukázať na príklade značení tried:

In [ ]:
class_mapping = {
    label: idx for idx, label in
    enumerate(np.unique(df['classlabel']))
}
df['classlabel'] = df['classlabel'].map(class_mapping)
df

Pre úpravu očakávaných labelov sa často používa aj `LabelEncoder` zo `scikit-learn`:

In [ ]:
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y

Pre nominálne hodnoty je potrebné vykonať ešte jeden krok, a to **one-hot encoding**, kde nominálne príznaky reprezentujeme ako vektor s jedinou hodnotou 1 (na ostatných pozíciách máme 0).

In [ ]:
from sklearn.preprocessing import OneHotEncoder
X = df[['color', 'size', 'price']].values
color_ohe = OneHotEncoder()
color_ohe.fit_transform(X[:, 0].reshape(-1, 1)).toarray()

**Poznámka:** Dávajte si pozor na to, že one-hot encoding často vytvára kolineárne vektory, kde jednotlivé atribúty silne korelujú, práve preto sa niekedy vynechá jedna hodnota zo získaného vektora.

In [ ]:
pd.get_dummies(df[['price', 'color', 'size']], drop_first=True)  # get_dummies works on string values only

## Škálovanie hodnôt

Cieľom škálovania je upraviť reprezentáciu príznakov tak, aby sa hodnoty jednotlivých príznakov pohybovali v intervale 0-1 so štandardnou deviáciou 1. To umožní lepšiu klasifikáciu príkladov. Ako už bolo vysvetlené na predchádzajúcich cvičeniach, takúto normalizáciu vždy vykonávame iba na základe trénovacích údajov. Najčastejšie prístupy sú Min-Max škálovanie a škálovanie na základe priemeru.

In [ ]:
df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', header=None)
df_wine.columns = [
    'Class label', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash',
    'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavanoid phenols',
    'Proanthocyanins', 'Color intensity', 'Hue',
    'OD280/OD315 of diluted wines', 'Proline'
]
df_wine.head()

In [ ]:
from sklearn.model_selection import train_test_split
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X_train_norm = mms.fit_transform(X_train)
X_test_norm = mms.transform(X_test)

print(X_train_norm[:5])
print(X_test_norm[:5])

In [ ]:
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

print(X_train_std[:5])
print(X_test_std[:5])

## Výber príznakov

Ak model má k dispozícii veľa príznakov pre jednotlivé príklady, môže dôjsť k pretrénovaniu, keďže sa model príliš sústreďuje na priame súvislosti medzi niektorým atribútom a výsledkom predikcie v trénovacích dátach, ktoré však nemusia platiť pre testovacie dáta. Možným riešením pretrénovania je samozrejme regularizácia, ktorú sme preberali na predošlom cvičení, v niektorých prípadoch však môže pomôcť aj zmenšenie dimenzionality údajov a to konkrétne dvomi prístupmi: výberom príznakov a extrakciou príznakov. Pod výberom príznakov rozumieme selekciu niektorých atribútov, ktoré neskôr použijeme pri predikcii, pričom cieľom extrakcie je vyťažiť zo všetkých príznakov charakteristické črty, ktoré lepšie popisujú príklady. Výsledkom výberu je teda zmenšený priestor príznakov, pričom extrakcia nám vytvorí úplne nový priestor príznakov.

### Sekvenčný výber príznakov

Cieľom sekvenčného výberu príznakov je zmenšiť dimenzionalitu príznakového priestoru na najrelevantnejšie príznaky. Tým sa odstráni šum, zvýšia sa generalizačné schopnosti modelu a zlepší sa výpočtová efektivita. Príkladom je algoritmu **sequential backward selection**, ktorý primárne slúži na zlepšenie efektivity. Algoritmus po jednom odstráni príznaky, až kým nedosiahneme ich cieľový počet. Robí to na základe kriteriálnej funkcie *J*, ktorú chceme minimalizovať (napríklad rozdiel medzi výkonom klasifikátora pred a po odstránení príznaku).

### Vyhodnotenie dôležitosti príznakov

Pre zníženie počtu príznakov pri lineárne separabilnom probléme môžeme používať kombináciu L1 regularizácie a lineárnej regresie. V prípade nelinearít však táto metóda je nepoužiteľná, vtedy môžeme použiť napríklad **random forest** na určenie dôležitosti jednotlivých príznakov. Dôležitosť sa vypočíta z priemernej miery entropie jednotlivých rozhodovacích stromov.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

feat_labels = df_wine.columns[1:]
forest = RandomForestClassifier(n_estimators=500, random_state=1)
forest.fit(X_train, y_train)
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]

plt.title('Feature importance')
plt.bar(range(X_train.shape[1]), importances[indices], align='center')
plt.xticks(range(X_train.shape[1]), feat_labels[indices], rotation=90)
plt.xlim([-1, X_train.shape[1]])
plt.tight_layout()
plt.show()

### Principal Component Analysis

**Principal Component Analysis** je základnou metódou extrakcie príznakov, ktorá okrem redukcie dimenzionality príznakového priestoru sa používa aj na dátovú analýzu a odstránenie šumu. PCA dokáže odhaliť súvislosti a vzory v dátach na základe korelácie medzi príznakmi. Takáto analýza odhalí smer najväčšej variancie.

PCA reálne vygeneruje transformačnú maticu $d \times k$, ktorú vieme použiť na mapovanie *d*-dimenzionálneho príkladu do *k* dimenzií. Postup pri analýze je nasledovný:

1. **Štandardizujte *d*-dimenzionálny dataset**.
2. **Vytvorte maticu kovariancie**.
3. **Rozložte maticu kovariancie na vlastné vektory a vlastné čísla**.
4. **Usporiadajte vlastné čísla zostupne, aby ste zoradili príslušné vlastné vektory**.
5. **Vyberte *k* vlastných vektorov**, ktoré zodpovedajú *k* najväčším vlastným číslam, kde *k* je dimenzionalita novej podmnožiny príznakov.
6. **Vytvorte projekčnú maticu W z top *k* vlastných vektorov**.
7. **Transformujte *d*-dimenzionálny vstupný dataset *X* pomocou projekčnej matice *W*** na získanie novej *k*-dimenzionálnej podmnožiny príznakov.

In [ ]:
from sklearn.decomposition import PCA

X_train_pca = pca.fit_transform(X_train_std)
X_test_pca = pca.transform(X_test_std)

Príklad transformácie klasifikačnej úlohy pomocou PCA:

![](lab06/pca1.jpg)

![](lab06/pca2.jpg)

### Linear Discriminant Analysis

Kým PCA hľadá osi maximálnej variancie, cieľom **Linear Discriminant Analysis** je nájsť podmnožinu (extrahovaných) príznakov, ktorá maximalizuje separabilitu tried. LDA tým pádom na rozdiel od PCA predstavuje kontrolovaný algoritmus, ktorý berie do úvahy cieľové triedy. LDA tiež pracuje s niekoľkými predpokladmi, ako normálne rozdelenie tried, identické kovariančné matice tried a štatistická nezávislosť príkladov. Tieto predpoklady však nie sú striktné, algoritmus pomerne dobre funguje aj keď niektorý z nich je porušený.

1. **Štandardizujte *d*-dimenzionálny dataset** (kde *d* je počet príznakov).
2. **Pre každú triedu vypočítajte *d*-dimenzionálny priemerný vektor**. (kontrolovaná časť algoritmu)
3. **Vytvorte maticu rozptylu medzi triedami, $S_B$, a maticu rozptylu v rámci tried, $S_W$**.
4. **Vypočítajte vlastné vektory a príslušné vlastné čísla matice $S_W^{-1} S_B$**.
5. **Usporiadajte vlastné čísla zostupne, aby ste zoradili príslušné vlastné vektory**.
6. **Vyberte *k* vlastných vektorov**, ktoré zodpovedajú *k* najväčším vlastným číslam, a vytvorte transformačnú maticu $W$ s rozmermi *d×k*; vlastné vektory tvoria stĺpce tejto matice.
7. **Projekcia príkladov na novú množinu vlastností** pomocou transformačnej matice $W$.

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

lda = LDA(n_components=2)
X_train_lda = lda.fit_transform(X_train_std, y_train)

## Krížová validácia

V *k*-násobnej krížovej validácii náhodne rozdelíme trénovacie dáta na *k* častí bez náhrady. Z týchto častí sa *k – 1* použije na trénovanie modelu (*training fold*) a jedna časť (*test fold*) na vyhodnotenie výkonnosti. Tento postup sa opakuje *k*-krát, čím získame *k* modelov a k odhadov výkonnosti. Výkonnosť modelov je vypočítaná z priemeru týchto modelov. Ak máte dostatočné množstvo údajov, vyhodnotenie môžete robiť na rovnakých testovacích dátach, ktoré nevstupujú do krížovej validácie, alebo môžete použiť výkonnosť modelov na testovacích foldoch. Krížová validácia sa v druhom prípade zvyčajne používa na optimalizáciu hyperparametrov.

Najčastejšie sa používa 10-násobná krížová validácia, keďže tá sa ukázala byť najreprezentatívnejšie a najviac balansuje bias-variance tradeoff. Čím je dataset menší, tím viac foldov by sme mali použiť, pri väčších datasetoch sa často používa *k = 5*. Ak máte nevybalansovaný dataset, môžete používať stratifikovanú krížovú validáciu.

In [ ]:
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=10).split(X_train, y_train)
for k, (train, test) in enumerate(kfold):
    # train your model: fit(X_train[train], y_train[train]) 
    # evaluate your model: score(X_train[test], y_train[test])
    pass


# from sklearn.model_selection import cross_val_score
# scores = cross_val_score(estimator=pipe_lr, X=X_train, y=y_train, cv=10, n_jobs=1)